In [ ]:
import imgTransformations
import utils_jupyter
import utils

import os

import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
#initilize path for images
dirname = os.path.abspath('')
folder = os.path.join(dirname, 'MEFDatabase/source image sequences/')
images = [] 

# Pass the images list to the interactive selector
utils_jupyter.interactive_image_selector(folder, images)

Definisci qua i metodi e inseriscili nel dizionario qua sotto, NON CAMBIARE I NOMI

In [ ]:
methods = {
    'Average Fusion': imgTransformations.average_fusion,
    'Mertens Fusion': imgTransformations.mertens_fusion,
    'Laplacian Pyramid': imgTransformations.laplacian_pyramid_fusion,
    'Exposure Fusion' : imgTransformations.exposure_fusion,
    'Exposure Compensation': imgTransformations.exposure_compensation_fusion,
    'Enhanced Exposure': imgTransformations.enhanced_exposure_fusion,
    'Domain Transform': imgTransformations.domain_transform_fusion,
    'Domain Transform - Homebrew': imgTransformations.domain_transform_fusion,
    'Wavelet Fusion': imgTransformations.wavelet_fusion
}

#### Interactive MEF Method Explorer

This widget showcases Multi-Exposure Fusion (MEF) techniques with real-time parameter tuning.
Features tabbed interfaces for 8 fusion methods, including Laplacian Pyramid, Domain Transform, and Wavelet Fusion. Adjust sliders to optimize sigma/epsilon/levels and instantly visualize results with fused images and RGB histograms. Designed for rapid comparison of fusion outputs.

In [ ]:
utils_jupyter.showcase_methods_tab(images, methods)

#### Domain Transform Filter  
A edge-preserving smoothing technique that:  
- Treats the image as a 3D surface (x,y,intensity)  
- Performs anisotropic diffusion along intensity gradients  
- Controlled by two parameters:  
  - σₛ (spatial): Controls geometric smoothness (60px in this implementation)  
  - σ꜀ (color): Preserves edges with similar intensity (0.4 = ±102/255 intensity tolerance)  

### Weight Map Calculation for Exposure Fusion  
`calculate_weight_maps(img)` computes pixel-wise fusion weights using three quality metrics:  

1. **Contrast** (∇²): Calculated via Laplacian operator on grayscale intensity, highlighting edges and textures where 
```math 
L(x,y) = |∂²I/∂x² + ∂²I/∂y²|
``` 

2. **Saturation**: Measured as standard deviation across RGB channels, favoring vibrant colors  
 ```math
 σ = √(Σ(Iᵢ - μ)²/3)
 ```

3. **Well-exposedness**: Gaussian probability that penalizes over/under-exposed pixels (μ=0.5 represents ideal mid-tone exposure)  
```math
 exp(-(I-0.5)²/(2×0.2²)
 ```

The combined weight map is computed as:  
```math
W(x,y) = (L(x,y)+ε) × (σ(x,y)+ε) × (E(x,y)+ε)
```
where ε=1e⁻⁶ ensures numerical stability during normalization.  

The function returns:  
1. Raw weight map  
2. Homebrew DT-filtered version (iterative bilateral filtering approximation)  
3. OpenCV's optimized DT-filter result  




In [ ]:
def calculate_weight_maps(img):
    epsilon = 1e-6

    guidance = img.astype(np.float32) / 255.0

    gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    contrast = np.abs(cv2.Laplacian(gray, cv2.CV_32F))

    saturation = np.std(img, axis=2)

    well_exposedness = np.exp(-0.5 * ((img - 0.5) / 0.2) ** 2)
    well_exposedness = np.prod(well_exposedness, axis=2)

    weight = (contrast + epsilon) * (saturation + epsilon) * (well_exposedness + epsilon)

    # Apply Homebrew Domain Transform filter
    smooth_weight_homebrew = imgTransformations.dt_filter_homebrew(guidance, weight, sigmaSpatial=60, sigmaColor=0.4, num_iterations=2)

    # Apply OpenCV's dtFilter
    smooth_weight_opencv = cv2.ximgproc.dtFilter(img, weight.astype(np.float32), sigmaSpatial=60, sigmaColor=0.4, mode=1)
    
    return weight, smooth_weight_homebrew, smooth_weight_opencv


utils_jupyter.showcase_weight_maps_tab(images, calculate_weight_maps)



In [ ]:
#comparison between homebrew and opencv dt filter
title, fused = methods["Domain Transform"](images, homebrew_dt=False)
title_hb, fused_hb = methods["Domain Transform"](images, homebrew_dt=True)

plt.figure(figsize=(18, 5))
plt.subplot(1, 2, 1)
plt.imshow(fused)
plt.title(title)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(fused_hb)
plt.title(title_hb)
plt.axis('off')

### Image Quality Analysis Functions

### 1. Entropy Calculation

Measures the **information content** in an image using Shannon entropy. For each RGB channel:

1. **Histogram Creation**:  
   Counts pixel intensity occurrences across 256 bins (0-255 range).

2. **Probability Distribution**:  
   Converts counts to probabilities:  
   ```math
   p(i) = \frac{\text{count}(i)}{\text{total pixels}}
   ```

3. **Entropy Calculation**:  
   Computes average surprise per pixel:  
   ```math
   H = -\sum_{i=0}^{255} p(i) \cdot \log_2 p(i)
   ```  
   (Zero probabilities are excluded to avoid mathematical errors)

The final result averages entropy values across all three color channels.

---

### 2. Spatial Frequency Calculation
Quantifies **edge activity** through gradient analysis. For each RGB channel:

1. **Horizontal Variation (RF)**:  
   Measures row-wise differences:  
   ```math
   RF = \sqrt{\frac{1}{MN} \sum_{x=1}^{M-1} \sum_{y=0}^{N-1} [I(x,y) - I(x-1,y)]^2}
   ```

2. **Vertical Variation (CF)**:  
   Measures column-wise differences:  
   ```math
   CF = \sqrt{\frac{1}{MN} \sum_{x=0}^{M-1} \sum_{y=1}^{N-1} [I(x,y) - I(x,y-1)]^2}
   ```

3. **Combined Metric**:  
   Merges directions using vector magnitude:  
   ```math
   SF = \sqrt{RF^2 + CF^2}
   ```

The final output averages spatial frequency across all channels.

---

### Key Characteristics Comparison

| Aspect              | Entropy                          | Spatial Frequency               |
|---------------------|----------------------------------|----------------------------------|
| **What it measures** | Randomness in pixel values      | Intensity changes between pixels |
| **High values mean** | Complex textures/variation      | Sharp edges/strong transitions   |
| **Mathematical base**| Information theory              | Gradient energy analysis         |
| **Typical range**   | 0-8 bits (8bpp images)          | 0-50 (image size dependent)      |
| **Usage scenario**  | Assessing texture preservation  | Evaluating edge sharpness        |


In [ ]:
def calculate_entropy(image):
    """Calculates entropy for an RGB image by averaging entropy across channels."""
    entropy_values = []
    for channel in range(image.shape[-1]):  # Loop through R, G, B channels
        hist, _ = np.histogram(image[:, :, channel].ravel(), bins=256, range=(0, 256))
        hist = hist.astype(np.float32) / hist.sum()
        hist = hist[hist > 0]  # Remove zero probabilities
        entropy_values.append(-np.sum(hist * np.log2(hist)))
    return np.mean(entropy_values)  # Average entropy across channels

def spatial_frequency(image):
    """Calculates spatial frequency for an RGB image."""
    sf_values = []
    for channel in range(image.shape[-1]):  # Compute SF for each color channel
        rows, cols = image.shape[:2]
        row_freq = np.sqrt(np.sum(np.diff(image[:, :, channel], axis=0) ** 2) / (rows * cols))
        col_freq = np.sqrt(np.sum(np.diff(image[:, :, channel], axis=1) ** 2) / (rows * cols))
        sf_values.append(np.sqrt(row_freq**2 + col_freq**2))
    return np.mean(sf_values)  # Average SF across channels

## Understanding Similarity Metrics: SSIM and PSNR

### Overview

The function `calculate_metrics_comparison` compares two images using two popular similarity metrics:
- **SSIM (Structural Similarity Index Measure)**
- **PSNR (Peak Signal-to-Noise Ratio)**

These metrics help quantify how similar two images are, which is particularly useful in image processing tasks such as image compression, restoration, and quality assessment.

### 1. SSIM (Structural Similarity Index Measure)

#### Mathematical Explanation
SSIM measures the perceptual similarity between two images by comparing their luminance, contrast, and structural information. Although the complete formula is complex, a simplified version is:

```math
SSIM(x, y) = \frac{(2 \mu_x \mu_y + C_1)(2 \sigma_{xy} + C_2)}{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)}
```

where:
- $\mu_x $ and $ \mu_y $ are the mean intensities of images `x` and `y`.
- $ \sigma_x^2 $ and $ \sigma_y^2 $ are the variances of `x` and `y`.
- $ \sigma_{xy} $ is the covariance between `x` and `y`.
- $ C_1 $ and $ C_2 $ are constants to stabilize the division when denominators are close to zero.

### 2. PSNR (Peak Signal-to-Noise Ratio)

#### Mathematical Explanation
PSNR measures the quality of a reconstructed image compared to its original version. It is calculated using the Mean Squared Error (MSE) between the images. The formula for PSNR is:

```math
PSNR = 20 \cdot \log_{10}(MAX_I) - 10 \cdot \log_{10}(MSE)
```

where:
- $ MAX_I $ is the maximum possible pixel value (255 for 8-bit images).
- **MSE** is the Mean Squared Error between the two images, defined as:

```math
MSE = \frac{1}{M \times N} \sum_{x,y} \left[ I_1(x, y) - I_2(x, y) \right]^2
```

If the MSE is zero (meaning the images are identical), PSNR is set to infinity.

### Process of the Algorithm

1. **Input Validation and Preprocessing:**
   - **Dimension Check:** The function first verifies whether the two images have the same shape. If not, it resizes the second image to match the dimensions of the first.
   - **Grayscale Conversion:** Both images are converted from RGB to grayscale, as SSIM and PSNR are typically calculated on single-channel images.

2. **SSIM Calculation:**
   - The SSIM metric is computed using the grayscale images to assess the structural similarity between them.

3. **PSNR Calculation:**
   - **MSE Calculation:** The Mean Squared Error between the two grayscale images is computed.
   - **PSNR Formula:** If the MSE is zero, PSNR is set to infinity. Otherwise, PSNR is calculated using the logarithmic formula based on the MSE.

4. **Output:**
   - The function returns two values: the SSIM value (`ssim_value`) and the PSNR value (`psnr_value`).

### Conclusion

- **SSIM** provides a measure of the perceptual similarity between images by taking into account luminance, contrast, and structural differences.
- **PSNR** quantifies the error between images, with higher values indicating better image quality.
- Together, these metrics are essential for comparing image quality and guiding improvements in image processing tasks.


In [ ]:
def calculate_metrics_comparison(img1, img2):
    """Calculate similarity metrics between two images"""
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    # Convert to grayscale for SSIM/PSNR
    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    
    # Calculate SSIM
    ssim_value = ssim(gray1, gray2, data_range=255)
    
    # Calculate PSNR with error handling
    mse = np.mean((gray1 - gray2) ** 2)
    if mse == 0:
        psnr_value = np.inf
    else:
        with np.errstate(divide='ignore'):
            psnr_value = 20 * np.log10(255) - 10 * np.log10(mse)
    
    return ssim_value, psnr_value

utils_jupyter.compare_methods_and_metrics(spatial_frequency, calculate_entropy, calculate_metrics_comparison, images, methods)

## Comparing Images with Heatmaps and RGB Histograms

This snippet performs the following tasks:

- **Compute Difference:**  
  Calculates the absolute per-pixel difference between two images to highlight changes.

- **Generate Heatmap:**  
  Converts the difference image to grayscale and displays it as a heatmap using a "hot" color map to visualize intensity differences.

- **Display Comparisons:**  
  Uses a utility function to compare multiple images by showing their difference heatmaps alongside their RGB histograms, providing insights into color distribution and changes.

This approach is useful for tasks like image quality assessment and change detection.


In [ ]:
def compute_difference(image1, image2):
    """Compute absolute difference between two images."""
    return cv2.absdiff(image1, image2)

def show_heatmap(diff_image):
    """Display a heatmap of the differences."""
    diff_gray = cv2.cvtColor(diff_image, cv2.COLOR_RGB2GRAY)
    plt.imshow(diff_gray, cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.title("Difference Heatmap")
    plt.show()

utils_jupyter.compare_methods_display(images, compute_difference, show_heatmap, methods)